In [59]:
from utils2d import check_tensorflow_gpu, check_torch_gpu
check_torch_gpu()

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat, savemat
from cv2 import resize

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: Python39 | Python version: 3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]
Torch version: 2.0.0+cu117
Torch build with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080



In [57]:
# Define custom dataset classes for train and test sets
class CustomDataset(Dataset):
    def __init__(self, root_X, root_y, filenames_X, filenames_y):
        self.root_X, self.root_y = root_X, root_y
        self.filenames_X = filenames_X
        self.filenames_y = filenames_y

    def __len__(self):
        return len(self.filenames_X)

    def __getitem__(self, idx):
        X_path = os.path.join(self.root_X, self.filenames_X[idx])
        y_path = os.path.join(self.root_y, self.filenames_y[idx])
        X_array, y_array = np.load(X_path), np.load(y_path)
        X_normalized = (X_array - X_array.min()) / (X_array.max() - X_array.min())
        y_normalized = (y_array - y_array.min()) / (y_array.max() - y_array.min())
        X_tensor = torch.from_numpy(X_normalized)
        y_tensor = torch.from_numpy(y_normalized)
        return X_tensor, y_tensor

In [58]:
# Define your root folders
root_X_folder, root_y_folder = 'simulations2D/input_features', 'simulations2D/output_targets'
X_filenames,   y_filenames   =  os.listdir(root_X_folder),      os.listdir(root_y_folder)
X_train_filenames, X_test_filenames, y_train_filenames, y_test_filenames = train_test_split(X_filenames, y_filenames, test_size=0.25, random_state=42)

# Create custom dataset instances for train and test sets
train_dataset = CustomDataset(root_X=root_X_folder, root_y=root_y_folder, filenames_X=X_train_filenames, filenames_y=y_train_filenames)
test_dataset  = CustomDataset(root_X=root_X_folder, root_y=root_y_folder, filenames_X=X_test_filenames,  filenames_y=y_test_filenames)

# Create dataloaders for train and test sets
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

In [76]:
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), bias=False):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels,  kernel_size=kernel_size, groups=in_channels, bias=bias, padding=1)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [77]:
class convBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(convBlock, self).__init__()
        self.conv = SeparableConv2d(in_ch, out_ch)
        self.norm = nn.InstanceNorm2d(out_ch)
        self.actv = nn.LeakyReLU(0.3)
        self.pool = MaxPool2d(2)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.actv(x)
        x = self.pool(x)
        return x


In [78]:
class proxy(nn.Module):
    def __init__(self):
        super(proxy, self).__init__()
        self.conv1 = convBlock(4, 16)
        self.conv2 = convBlock(16, 32)
        self.conv3 = convBlock(32, 64)
        self.conv4 = convBlock(64, 128)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x

In [80]:
model = proxy()
model(x)

NameError: name 'MaxPool2d' is not defined